<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Convert NFTD to COGs

_by Jeremy Freeman (CarbonPlan), August 1, 2020_

This notebook converts local copies of NFTD datasets to cloud optimized
GeoTIFFs.

**Inputs:**

- downloaded files from UFSF raster gateway

**Outputs:**

- Cloud optimized GeoTIFFs

**Notes:**

- No reprojection or processing of the data is done in this notebook.


In [ ]:
import io
import os
import pathlib

from google.cloud import storage
from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

In [ ]:
storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB

In [ ]:
dst_profile = cog_profiles.get("deflate")
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "/Users/freeman/.config/gcloud/legacy_credentials/jeremy@carbonplan.org/adc.json"


def upload(src, target, bucket="carbonplan-data"):
    storage_client = storage.Client("carbonplan")
    bucket = storage_client.bucket(bucket)
    blob = bucket.blob(target)
    blob.upload_from_filename(src)

## Iterate over datasets


In [ ]:
# conus forest group
workdir = pathlib.Path("/Users/freeman/workdir/carbonplan_data_downloads/nftd/")
source = workdir / "conus_forestgroup" / "conus_forestgroup.img"
target = workdir / "conus_forestgroup" / "raster.tif"
cloud = "raw/nftd/conus_forestgroup/30m/raster.tif"

In [ ]:
files = [
    {
        "source": workdir / "conus_forestgroup" / "conus_forestgroup.img",
        "target": workdir / "conus_forestgroup" / "raster.tif",
        "cloud": "raw/nftd/conus_forestgroup/250m/raster.tif",
    },
    {
        "source": workdir / "conus_forestgroup" / "conus_forestgroup_error.img",
        "target": workdir / "conus_forestgroup" / "error.tif",
        "cloud": "raw/nftd/conus_forestgroup/250m/error.tif",
    },
    {
        "source": workdir / "conus_forest-type" / "conus_foresttype.img",
        "target": workdir / "conus_forest-type" / "raster.tif",
        "cloud": "raw/nftd/conus_foresttype/250m/raster.tif",
    },
    {
        "source": workdir / "conus_forest-type" / "conus_foresttype_error.img",
        "target": workdir / "conus_forest-type" / "error.tif",
        "cloud": "raw/nftd/conus_foresttype/250m/error.tif",
    },
    {
        "source": workdir / "ak_forestgroup" / "alaska_forestgroup_63360.img",
        "target": workdir / "ak_forestgroup" / "raster.tif",
        "cloud": "raw/nftd/ak_forestgroup/250m/raster.tif",
    },
    {
        "source": workdir
        / "ak_forestgroup"
        / "alaska_forestgroup_confidence_63360.img",
        "target": workdir / "ak_forestgroup" / "error.tif",
        "cloud": "raw/nftd/ak_forestgroup/250m/error.tif",
    },
    {
        "source": workdir / "ak_forest-type" / "alaska_foresttype_63360.img",
        "target": workdir / "ak_forest-type" / "raster.tif",
        "cloud": "raw/nftd/ak_foresttype/250m/raster.tif",
    },
    {
        "source": workdir
        / "ak_forest-type"
        / "alaska_foresttype_confidence_63360.img",
        "target": workdir / "ak_forest-type" / "error.tif",
        "cloud": "raw/nftd/ak_foresttype/250m/error.tif",
    },
]

In [ ]:
# convert to cogs
[cog_translate(f["source"], f["target"], dst_profile) for f in files]

In [ ]:
# upload to cloud storage
[upload(f["target"], f["cloud"]) for f in files]